# Running Hogtie Using different genealogies

In [1]:
import ipcoal
import toytree
import os
import numpy as np
import pandas as pd

In [2]:
#get a random tree
tree = toytree.rtree.imbtree(ntips=10, treeheight=1e5)
tree.draw(ts='p')

(<toyplot.canvas.Canvas at 0x7f940944aee0>,
 <toytree.Render.ToytreeMark at 0x7f940cdaf310>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="260.0px" viewBox="0 0 350.0 260.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tfe47c49a1bf543dab93d81f675d4ec23"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 r0 r1 r2 r3 r4 r5 r6 r7 r8 r9 0 50000 100000

In [5]:
#create model
mod_introgress =  ipcoal.Model(tree=tree, Ne=1e5, admixture_edges=[(3, 8, 0.5, 0.5)], nsamples=1)
mod_introgress.sim_loci(nloci=1, nsites=1000) #1 haploid chromosome
genos_introgress=mod_introgress.write_vcf() #i need a matrix of 1's and 0's
genos_introgress

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,r0,r1,r2,r3,r4,r5,r6,r7,r8,r9
0,0,27,.,T,C,99,PASS,.,GT,0,1,1,0,1,0,0,0,0,0
1,0,43,.,A,C,99,PASS,.,GT,1,0,0,0,0,0,0,0,0,0
2,0,54,.,A,G,99,PASS,.,GT,1,0,0,0,0,1,1,1,0,0
3,0,74,.,G,C,99,PASS,.,GT,1,0,0,0,0,1,1,1,0,0
4,0,91,.,T,C,99,PASS,.,GT,0,1,1,0,1,0,0,0,0,0
5,0,109,.,G,T,99,PASS,.,GT,1,0,0,0,0,1,0,1,0,0
6,0,172,.,G,T,99,PASS,.,GT,0,0,0,0,0,0,1,0,0,0
7,0,198,.,G,A,99,PASS,.,GT,0,1,1,0,1,0,0,0,0,0
8,0,394,.,T,A,99,PASS,.,GT,0,1,1,0,1,0,0,0,0,0
9,0,476,.,G,T,99,PASS,.,GT,1,1,1,0,1,1,1,1,0,0


In [6]:
mod_introgress.df.iloc[1, 2]

1000

In [3]:
#works for simulated loci, not snps
HOGTIEDIR = os.path.dirname(os.getcwd())
def genealogy_try2(model):
    """
    model must be an ipcoal model object
    objective: make a unique dataframe for each genealogy with the sites that follow
    that genealogy
    
    TO DO: concatenate the reordered dataframes into one big dataframe that can be run through MatrixParser
    """
    vcf = model.write_vcf()
    
    dataframe = 0
    for idx in model.df.index:
        dataframe += 1
        start = model.df.iloc[idx, 1]
        end = model.df.iloc[idx ,2]
        gen = toytree.tree(model.df.iloc[idx, 6], tree_format=0)
        
        count = 0
        df = pd.DataFrame()
        for row in vcf.index:
            count += 1
            if start < vcf.iloc[row, 1] < end:
                df[f'{count}'] = vcf.iloc[row, 9:]
        df = df.reindex(gen.get_tip_labels())
        #file = os.path.join(HOGTIEDIR, "sampledata", f"genealogy{dataframe}.csv")
        #df.to_csv(file)

In [8]:
genealogy_try2(mod_introgress)

In [79]:
test.reorder()
test.df

,1,2,3,4,5,6,7
r0,0,0,0,0,0,1,0
r1,0,0,0,0,0,0,0
r2,0,0,0,1,0,0,0
r3,0,1,1,0,1,0,0
r4,0,0,0,0,0,0,0
r5,0,0,0,0,0,0,0
r6,0,0,0,0,0,0,1
r7,1,0,0,0,0,1,0
r8,0,0,0,0,0,1,0
r9,0,0,0,0,0,0,0


In [70]:
gen = toytree.tree(mod_introgress.df.iloc[0, 6], tree_format=0)
print(gen.get_tip_labels())
data.reindex(gen.get_tip_labels())

['r8', 'r7', 'r0', 'r2', 'r4', 'r1', 'r5', 'r6', 'r9', 'r3']


,1,2,3,4,5,6,7
r8,0,0,0,0,0,1,0
r7,1,0,0,0,0,1,0
r0,0,0,0,0,0,1,0
r2,0,0,0,1,0,0,0
r4,0,0,0,0,0,0,0
r1,0,0,0,0,0,0,0
r5,0,0,0,0,0,0,0
r6,0,0,0,0,0,0,1
r9,0,0,0,0,0,0,0
r3,0,1,1,0,1,0,0


## Pair re-ordered columns with genealogies to be run through hogtie's DiscreteMarkovModel

In [62]:
mod_snps =  ipcoal.Model(tree=tree, Ne=1e5, admixture_edges=[(3, 8, 0.5, 0.5)], nsamples=1)
mod_snps.sim_snps(nsnps=10)
mod_snps.write_vcf().iloc[:,9:].T

,0,1,2,3,4,5,6,7,8,9
r0,1,1,0,0,0,0,0,0,0,0
r1,0,1,0,0,0,0,0,0,0,0
r2,0,1,0,0,0,0,0,0,0,0
r3,0,0,0,0,0,0,0,1,0,0
r4,1,1,0,1,0,0,1,0,0,1
r5,1,1,0,0,0,1,0,0,0,1
r6,0,0,0,0,0,0,0,0,0,0
r7,1,1,1,0,1,0,0,0,0,0
r8,0,0,0,0,0,0,1,0,1,0
r9,0,1,1,0,0,0,0,0,0,0


In [45]:
mod_snps.df

,locus,start,end,nbps,nsnps,tidx,genealogy
0,0,0,1,1,1,0,"((r8:109373,r9:109373):4..."
1,1,0,1,1,1,0,"((r6:115448,(r5:98988.1,..."
2,2,0,1,1,1,0,"(r7:400306,((r4:108911,(..."
3,3,0,1,1,1,0,"((r3:178770,((r1:60081.6..."
4,4,0,1,1,1,0,"(r6:389517,(r9:364632,(r..."
5,5,0,1,1,1,0,"(r6:610291,(r3:512158,(r..."
6,6,0,1,1,1,0,"(r8:381474,(r6:159190,(r..."
7,7,0,1,1,1,0,"(r3:306465,((r0:151614,(..."
8,8,0,1,1,1,0,"(r1:473872,(((r0:77808.7..."
9,9,0,1,1,1,0,"(((r2:101933,r9:101933):..."


In [9]:
from hogtie import DiscreteMarkovModel

In [10]:
TREE = toytree.rtree.baltree(ntips=12, treeheight=1e6)
MODEL = ipcoal.Model(TREE, Ne=20000, mut=1e-8, seed=123)
MODEL.sim_snps(10)
DATA = MODEL.write_vcf().iloc[:, 9:]
DATA[(DATA == 2) | (DATA == 3)] = 1
TREE_ONE = TREE.mod.node_scale_root_height(1)

In [12]:
TEST = DiscreteMarkovModel(TREE_ONE, DATA, 'ARD', prior=0.5)
TEST.optimize()

03:32 | DEBUG   | get_unique_data | uniq array shape: (8, 12)


In [52]:
from scipy.optimize import minimize
from scipy.linalg import expm

In [67]:
class NullLikeCalculations:
    """
    Calculates null likelihoods using optimized alpha and beta values
    """
    def __init__(self, tree, data, model, alpha, beta, prior=0.5):
        # store user inputs
        self.tree = tree
        self.data = data
        self.model = model
        self.prior_root_is_1 = prior
        # model parameters to be estimated (in ER model only alpha)
        
        # set to initial values based on the tree height units.
        self.alpha = alpha
        self.beta = beta
        self.log_lik = 0.
        
        # set likelihoods to 1 for data at tips, and None for internal
        self.set_initial_likelihoods()
        
        self.qmat = np.array([
                [-self.alpha, self.alpha],
                [self.beta, -self.beta]
               ])
        self.lik = 0
        
    def set_initial_likelihoods(self):
        """
        Sets the observed states at the tips as attributes of the nodes.
        """
        # get values as lists of [0, 1] or [1, 0]
        values = ([float(1 - i), float(i)] for i in self.data)
        # get range of tip idxs (0-ntips)
        keys = range(0, len(self.data))
        # map values to tips {0:x, 1:y, 2:z...}
        valuesdict = dict(zip(keys, values))
        # set as .likelihood attributes on tip nodes.
        self.tree = self.tree.set_node_values(
            feature="likelihood", 
            values=valuesdict,
            default=None,
        )
        
    def node_conditional_likelihood(self, nidx):
        """
        Returns the conditional likelihood at a single node given the
        likelihood's of data at its child nodes.
        """
        # get the TreeNode 
        node = self.tree.idx_dict[nidx]
        # get transition probabilities over each branch length
        prob_child0 = expm(self.qmat * node.children[0].dist)
        prob_child1 = expm(self.qmat * node.children[1].dist)
        # likelihood that child 0 observation occurs if anc==0
        child0_is0 = (
            prob_child0[0, 0] * node.children[0].likelihood[0] + 
            prob_child0[0, 1] * node.children[0].likelihood[1]
        )
        # likelihood that child 1 observation occurs if anc==0
        child1_is0 = (
            prob_child1[0, 0] * node.children[1].likelihood[0] + 
            prob_child1[0, 1] * node.children[1].likelihood[1]
        )
        anc_lik_0 = child0_is0 * child1_is0
        # likelihood that child 0 observation occurs if anc==1
        child0_is1 = (
            prob_child0[1, 0] * node.children[0].likelihood[0] + 
            prob_child0[1, 1] * node.children[0].likelihood[1]
        )
        child1_is1 = (
            prob_child1[1, 0] * node.children[1].likelihood[0] + 
            prob_child1[1, 1] * node.children[1].likelihood[1]
        )
        anc_lik_1 = child0_is1 * child1_is1
        # set estimated conditional likelihood on this node
        node.likelihood = [anc_lik_0, anc_lik_1]
        
    def pruning_algorithm(self):
        """
        Traverse tree from tips to root calculating conditional 
        likelihood at each internal node on the way, and compute final
        conditional likelihood at root based on priors for root state.
        """
        # traverse tree to get conditional likelihood estimate at root.
        for node in self.tree.treenode.traverse("postorder"):
            if not node.is_leaf():
                self.node_conditional_likelihood(node.idx)
        # multiply root prior times the conditional likelihood at root
        root = self.tree.treenode
        lik = (
            (1 - self.prior_root_is_1) * root.likelihood[0] + 
            self.prior_root_is_1 * root.likelihood[1]
        )
        self.lik = -np.log(lik)

In [68]:
def genealogy_reorder_snps(mod, model, alpha, beta, prior=0.5):
    """
    mod must be an ipcoal model object
    objective: make a unique dataframe for each genealogy with the sites that follow
    that genealogy
    
    TO DO: concatenate the reordered dataframes into one big dataframe that can be run through MatrixParser
    """
    liks = pd.DataFrame()
    vcf = mod.write_vcf().iloc[:,9:].T
    tree_list = []
    for idx in mod.df.index:
        genealogy = toytree.tree(mod.df.iloc[idx, 6], tree_format=0)
        tree_list.append(genealogy)
        
    likelihoods = np.empty((0,len(vcf.columns)),float)
    for col in vcf.columns:
        data = vcf[col]
        df = data.reindex(tree_list[col].get_tip_labels())
        
        null = NullLikeCalculations(tree_list[col], df, model=model, alpha=alpha, beta=beta, prior=prior)
        null.pruning_algorithm()
        
        likelihoods = np.append(likelihoods, null.lik)
    
    return likelihoods
        #liks['lik']=null.lik
    
    #return liks
        #file = os.path.join(HOGTIEDIR, "sampledata", f"genealogy{dataframe}.csv")
        #df.to_csv(file)

In [69]:
genealogy_reorder_snps(mod_snps, model='ARD', alpha=TEST.alpha, beta=TEST.beta)

array([13.95899968,  7.53495334, 18.24169724, 20.38304602, 20.38304602,
       20.38304602, 18.24169724, 20.38304602, 20.38304602, 18.24169724])